#### Question 4:

Consider the Movie Similarities code and problem that was discussed during the class (Session 4). Please provide a brief write-up on the problem, steps needed to arrive at the solution (recommendation system), and how exactly those steps are implemented in the code. While you are doing so, please also mention what each line of code does (It is not sufficient to mention what each block of code does, you would have to provide explanation for each line).

#### Answer:

Movie Similarities code is for item - item collaborating filtering based recommendation system for IMDB. Collaborative filtering is used to recommend new movie (item) based on the similarity of the users. This code calculates cosine similarity for the two movies. This code will recommend user to Top N similar movie for the given movie.

Item - item collaborating filtering look for movies that are similar to the movie that user has already rated and recommend most similar movies. 

Similarity between movies is decided by looking at how other users have rated them.

For Similarity computation, there are many method like Euclidian similarity, Cosine similarity and Correlation coefficient etc. But in this code we used Cosine similarity.

similarity = sum(rating1 . rating2) / sqrt(sum(rating1^2)) . sqrt(sum(rating2^2))

#### Amit Kumar Sharma
#### PGID: 71721079 


In [1]:
import os
import numpy as np

Load Data into data RDD.

In [2]:
os.getcwd()
data = sc.textFile("file:///Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data/u.data")

In [3]:
data.take(10)

[u'196\t242\t3\t881250949',
 u'186\t302\t3\t891717742',
 u'22\t377\t1\t878887116',
 u'244\t51\t2\t880606923',
 u'166\t346\t1\t886397596',
 u'298\t474\t4\t884182806',
 u'115\t265\t2\t881171488',
 u'253\t465\t5\t891628467',
 u'305\t451\t3\t886324817',
 u'6\t86\t3\t883603013']

As data in data RDD in tab separated format, make a tuple of (userId, (movieId, Rating)). Because we want to recommend similar movies which more than 2 users rated.

In [4]:
ratings= data.map(lambda x: x.split()).map(lambda x: (int(x[0]), (int(x[1]),float(x[2]))))
ratings.take(10)

[(196, (242, 3.0)),
 (186, (302, 3.0)),
 (22, (377, 1.0)),
 (244, (51, 2.0)),
 (166, (346, 1.0)),
 (298, (474, 4.0)),
 (115, (265, 2.0)),
 (253, (465, 5.0)),
 (305, (451, 3.0)),
 (6, (86, 3.0))]

Join rating rdd with rating rdd to find out all combination of movie and ratings.

In [5]:
joinedRatings= ratings.join(ratings)
joinedRatings.take(10)

[(512, ((265, 4.0), (265, 4.0))),
 (512, ((265, 4.0), (23, 4.0))),
 (512, ((265, 4.0), (1, 4.0))),
 (512, ((265, 4.0), (198, 5.0))),
 (512, ((265, 4.0), (318, 5.0))),
 (512, ((265, 4.0), (11, 5.0))),
 (512, ((265, 4.0), (1459, 4.0))),
 (512, ((265, 4.0), (313, 3.0))),
 (512, ((265, 4.0), (97, 5.0))),
 (512, ((265, 4.0), (191, 4.0)))]

Remove duplicate pairs from joinedRatings RDD. filterDuplicates method compare movie1 and movie2 and fetch only distict movies and rating.

In [6]:
def filterDuplicates( (userID, ratings) ):
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return movie1 < movie2

In [7]:
uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)
uniqueJoinedRatings.take(10)

[(512, ((265, 4.0), (318, 5.0))),
 (512, ((265, 4.0), (1459, 4.0))),
 (512, ((265, 4.0), (313, 3.0))),
 (512, ((265, 4.0), (527, 5.0))),
 (512, ((265, 4.0), (302, 4.0))),
 (512, ((265, 4.0), (273, 5.0))),
 (512, ((265, 4.0), (286, 5.0))),
 (512, ((265, 4.0), (325, 2.0))),
 (512, ((265, 4.0), (1238, 4.0))),
 (512, ((23, 4.0), (265, 4.0)))]

As we want to recommend similar movies for a given movie, that's why we don't need user information. Make pair of ((movie1, movie2), (rating1, rating2)) for each users instead of (user1, (movie1, rating1), (movie2, rating2)).

In [8]:
def makePairs((user, ratings)):
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))

In [9]:
# Now key by (movie1, movie2) pairs.
moviePairs = uniqueJoinedRatings.map(makePairs)
moviePairs.take(10)

[((265, 318), (4.0, 5.0)),
 ((265, 1459), (4.0, 4.0)),
 ((265, 313), (4.0, 3.0)),
 ((265, 527), (4.0, 5.0)),
 ((265, 302), (4.0, 4.0)),
 ((265, 273), (4.0, 5.0)),
 ((265, 286), (4.0, 5.0)),
 ((265, 325), (4.0, 2.0)),
 ((265, 1238), (4.0, 4.0)),
 ((23, 265), (4.0, 4.0))]

Group all the Ratings by two movies combination (movie1, movie2). This will make RDD of Keys (movie1, movie2) and list of ratings for both movies.

In [10]:
# We now have (movie1, movie2) => (rating1, rating2)
# Now collect all ratings for each movie pair and compute similarity
moviePairRatings = moviePairs.groupByKey()

In [11]:
moviePairRatings.filter(lambda x : x[0][0] == 265 and x[0][1] ==592).map(lambda x: (x[0], [i for i in x[1]])).take(10)

[((265, 592),
  [(5.0, 4.0),
   (5.0, 3.0),
   (3.0, 3.0),
   (5.0, 3.0),
   (2.0, 1.0),
   (5.0, 5.0),
   (5.0, 5.0)])]

#### Compute Similarity

Now we have data for all the 2 movies combination and their ratings for all the users. Now we can compute simalarity of these movies combination.

This computeCosineSimilarity method executes for all the movies combination and calculate similarities (co-relationship) between 2 movies. This method return a tuple of similartity score and no of occurance for similar movies combination in the dataset.

Formula for Similarity score is:<br>
similarity = sum(rating1 . rating2) / sqrt(sum(rating1^2)) . sqrt(sum(rating2^2))


Apply computeCosineSimilarity function on each movies pair and make moviePairSimilarities RDD.

In [12]:
def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = np.sqrt(sum_xx) * np.sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()

In [13]:
moviePairSimilarities.take(5)

[((197, 1097), (0.9758729093599599, 7)),
 ((42, 364), (0.9093486560398836, 18)),
 ((773, 1409), (1.0, 1)),
 ((273, 617), (0.9652953599007105, 7)),
 ((372, 974), (1.0, 1))]

#### loadMovieNames method:

This method is used to load u.item dataset and returns a dictionary of all the movie names and movie id. 

In [14]:
def loadMovieNames():
    movieNames = {}
    with open("/Users/a5sharma/Documents/ISB/BDM/HWAssignment/Assignment/data/u.item") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1].decode('ascii', 'ignore')
    return movieNames

nameDict = loadMovieNames()

Define threshold value of similary score and no of occurence.

In [15]:
scoreThreshold = 0.10
coOccurenceThreshold = 2

movieId : movieId is a Input argument. User can enter movieId in spark console.

Fetch all similar movies for the user's entered movieId and also, these similar movie's similarity score should greater than scoreThershold value. And Stored filtered result in filteredResults RDD.

In [16]:
# Extract similarities for the movie we care about that are "good".
#movieID = int(sys.argv[1])
movieID = int(input("Enter Movie Id: "))

# Filter for movies with this sim that are "good" as defined by
# our quality thresholds above
filteredResults = moviePairSimilarities.filter(lambda((pair,sim)): \
    (pair[0] == movieID or pair[1] == movieID) \
    and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold)

Enter Movie Id: 255


Fetch only 10 filtered results.

In [17]:
# Sort by quality score.
results = filteredResults.map(lambda((pair,sim)): (sim, pair)).sortByKey(ascending = False).take(10)

Display result of our recommendation system for user's entered movieid.

In [21]:
print "Top 10 similar movies for " + nameDict[movieID]
for result in results:
    (sim, pair) = result
    # Display the similarity result that isn't the movie we're looking at
    similarMovieID = pair[0]
    if (similarMovieID == movieID):
        similarMovieID = pair[1]
    print "Name: " + nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1])


Top 10 similar movies for My Best Friend's Wedding (1997)
Name: Kiss of Death (1995)	score: 1.0	strength: 4
Name: Cement Garden, The (1993)	score: 1.0	strength: 3
Name: Walking and Talking (1996)	score: 1.0	strength: 4
Name: Mute Witness (1994)	score: 1.0	strength: 3
Name: Poetic Justice (1993)	score: 0.999630382541	strength: 3
Name: Braindead (1992)	score: 0.999630382541	strength: 3
Name: Losing Isaiah (1995)	score: 0.999512076087	strength: 4
Name: Once Were Warriors (1994)	score: 0.997241374055	strength: 3
Name: Smile Like Yours, A (1997)	score: 0.996545758245	strength: 5
Name: Flower of My Secret, The (Flor de mi secreto, La) (1995)	score: 0.996272507219	strength: 3


In [19]:
nameDict[255]

u"My Best Friend's Wedding (1997)"